In [1]:
using Pkg, Revise
Pkg.activate("../GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, Random, LaTeXStrings, Latexify

Random.seed!(113312);

  Activating project at `C:\Users\Edgar Auger\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


<div style="float:center;width:100%;text-align: center;"><strong style="height:60px;color:darkred;font-size:40px;">Updating a Matrix Inverse</strong></div>

# 1. The Sherman-Morrison-Woodbury Formula

## 1.1. Derivation

Consider the block matrix $\begin{pmatrix} A & U \\ V & D \end{pmatrix}$, where $A$ and $D$ are square.

Assuming that $A^{-1}$ exists, we can eliminate $V$ and $U$<br>
$\qquad\begin{align}
\begin{pmatrix} A & U \\ V & D \end{pmatrix}
=& \begin{pmatrix} I & 0 \\ V A^{-1} & I \end{pmatrix} \begin{pmatrix} A & U \\ 0 & S_A \end{pmatrix} \\
=& \begin{pmatrix} I & 0 \\ V A^{-1} & I \end{pmatrix} \begin{pmatrix} A & 0 \\ 0 & S_A \end{pmatrix} \begin{pmatrix} I & A^{-1}U \\ 0 & I \end{pmatrix}
\end{align}$

where we have defined the **Schur Component** $S_A = D - V A^{-1} U$.

Further assuming that $A_A^{-1}$ exists, we can invert this equation to obtain<br>
$\qquad \begin{pmatrix} A & U \\ V & D \end{pmatrix}^{-1} =
 \begin{pmatrix} I & -A^{-1}U \\ 0 & I \end{pmatrix} 
 \begin{pmatrix} A^{-1} & 0 \\ 0 & S_A^{-1} \end{pmatrix} \begin{pmatrix} I & 0 \\ -V A^{-1} & I \end{pmatrix} \qquad\qquad\qquad\qquad (1)
$



We can similarly obtain a formula for this inverse assuming that $D^{-1}$ exists<br>

$\qquad \begin{pmatrix} A & U \\ V & D \end{pmatrix}^{-1} =
 \begin{pmatrix} I & 0 \\ -D^{-1}V & I \end{pmatrix} 
 \begin{pmatrix} S_D^{-1} & 0 \\ 0 & D^{-1} \end{pmatrix} \begin{pmatrix} I & - U D^{-1} \\ 0 & I \end{pmatrix} \qquad\qquad\qquad\qquad (2)
$

assuming the **Schur Component** $S_D = A - U D^{-1} V$ is also invertible.

Multiplying out the matrices in Eq 1 and Eq 2 and comparing terms, we obtain the Sherman-Morrison-Woodbury Formula<br>

$\qquad S_D^{-1} = A^{-1} + A^{-1} U S_A^{-1} V A^{-1} \Leftrightarrow \color{blue}{\left( A - U D^{-1} V \right)^{-1} = A^{-1} + A^{-1} U\; \left( D - V A^{-1} U \right)^{-1} \; V A^{-1} \qquad\qquad\qquad (3)}$

and<br>
$\qquad D S_A^{-1} V = V S_D^{-1} A$ 

#### Example

Consider the matrix $M = \left(\begin{array}{rr|r}
1 & -2 & -3 \\
-1 & 3 & 2 \\ \hline
-3 & 5 & 11
\end{array}\right),\;\;$
with
$A = \left(\begin{array}{rr} 1 & -2 \\ -1 & 3 \end{array}\right), \;\; U = \left(\begin{array}{r} -3 \\ 2 \end{array} \right),\;\; V = \left( -3\;\;5 \right)\;\;$ and $\;\;D= \left(11 \right)$

The Schur Components are $S_A = ( 1 )\;\;$ and $\;\; S_D = \frac{1}{11} \left( \begin{array}{rr} 2 & -7 \\ -5 & 23 \end{array}\right)$

In [2]:
# Check
M = [1 -2 -3; -1 3 2; -3 5 11]
A = M[1:2,1:2]; U = M[1:2,3:3]; V = M[3:3, 1:2]; D = M[3:3,3:3]
A_inv = inv(Rational{Int}.(A))
D_inv = inv(Rational{Int}.(D))
S_A = D - V * A_inv * U
S_D = A - U * D_inv * V

@show inv(A-U*D_inv*V) == (A_inv + A_inv*U *inv(S_A) * V*A_inv);

inv(A - U * D_inv * V) == A_inv + A_inv * U * inv(S_A) * V * A_inv = true


## 1.2. Rank 1 Update of a Matrix Inverse

Let $D = -I, U = u, V = v^t$ in the Sherman-Morrison-Woodbury Formula. We obtain<br>

$\qquad \begin{align} \left(A + u v^t \right)^{-1} &= A^{-1} - A^{-1} u\ (I+v^t A^{-1} u)^{-1}\ v^t A^{-1} \\
&= A^{-1} - \frac{1}{1+v^t A^{-1} u} A^{-1} u v^t A^{-1} \end{align}$



Given $A^{-1}$, the inverse of the rank 1 update $(A + u v^t)$ is obtained by a number of matrix multiplications<br>
$\qquad$ since $(I + v^t A^{-1} u)$ is a size $1 \times 1$ matrix that is trivially invertible.

#### Example

In [3]:
M = 5; k=1
A, A_inv = gen_inv_pb(M, maxint=2)
printstyled("Original Matrix A and its inverse", color=:blue, bold=true)
display(latexify([A', A_inv']'))

u        = 0.1*rand(M,1)
v        = rand(M,1)
uvt      = u*v'
printstyled("Rank 1 Update", color=:blue, bold=true)
display(latexify(round.(uvt, digits=3)))

alpha    = (I+v'A_inv*u)[1,1]
#inv( Real.( A+u*v' ))

printstyled("Updated Inverse", color=:blue, bold=true)
display(latexify(round.(A_inv - 1/alpha * (A_inv *u)*(v'*A_inv)),digits=3))

@show inv(A+uvt) ≈ (A_inv - 1/alpha * (A_inv *u)*(v'*A_inv));

Original Matrix A and its inverse

L"\begin{equation}
\left[
\begin{array}{cc}
\left[
\begin{array}{ccccc}
1 & 2 & 1 & 2 & 2 \\
-2 & -3 & -3 & -4 & -3 \\
-2 & -2 & -3 & -6 & -2 \\
-2 & -6 & -1 & -1 & -7 \\
2 & 6 & -2 & 8 & 7 \\
\end{array}
\right] & \left[
\begin{array}{ccccc}
51 & 52 & -27 & -8 & -8 \\
-2 & -7 & 5 & 2 & 1 \\
-14 & -14 & 7 & 2 & 2 \\
-6 & -6 & 3 & 1 & 1 \\
-10 & -6 & 2 & 0 & 1 \\
\end{array}
\right] \\
\end{array}
\right]
\end{equation}
"

Rank 1 Update

L"\begin{equation}
\left[
\begin{array}{ccccc}
0.044 & 0.022 & 0.019 & 0.03 & 0.048 \\
0.081 & 0.041 & 0.035 & 0.054 & 0.087 \\
0.015 & 0.008 & 0.007 & 0.01 & 0.016 \\
0.016 & 0.008 & 0.007 & 0.011 & 0.017 \\
0.014 & 0.007 & 0.006 & 0.01 & 0.015 \\
\end{array}
\right]
\end{equation}
"

Updated Inverse

L"\begin{equation}
\left[
\begin{array}{ccccc}
13.0 & 10.0 & -4.0 & -1.0 & -2.0 \\
1.0 & -3.0 & 3.0 & 1.0 & 0.0 \\
-4.0 & -3.0 & 1.0 & 0.0 & 0.0 \\
-2.0 & -1.0 & 0.0 & 0.0 & 0.0 \\
-4.0 & 0.0 & -1.0 & -1.0 & 0.0 \\
\end{array}
\right]
\end{equation}
"

inv(A + uvt) ≈ A_inv - (1 / alpha) * (A_inv * u) * (v' * A_inv) = true


## 1.3. Rank k Update of a Matrix Inverse

Consider updating an invertible matrix $A$ with a rank $k$ matrix $U V^t$, i.e.,<br>
$\qquad A + U V^t = A+u_1 v_1^t + \dots u_k v_k^t$ 

Let $D = -I$ in the Sherman-Morrison-Woodbury Formula, and name the matrix $V^t$ rather than $V$. We obtain<br>

$\qquad\qquad (A + U V^t)^{-1} = A^{-1} - A^{-1} U \left( I + V^t A^{-1} U \right)^{-1} V^t A^{-1},
$ provided the inverses exist.


Note that for $A \in \mathscr{R}^{N \times N},\;\;$ and $U, V \in \mathscr{R}^{N \times k}.$<br>
$\qquad\therefore\;\;$the matrix $I + V^t A^{-1} U$ has size $k \times k$.

The problem of inverting a size $N \times N$ matrix $A + U V^t$ is reduced to inverting a matrix of size $k \times k$.

In [4]:
M = 8; k=3
A, A_inv = gen_inv_pb(M, maxint=3)
_,U = gen_gj_matrix(M,k,k;maxint=2)
_,V = gen_gj_matrix(M,k,k;maxint=2)
printstyled( "A and A⁻¹", color=:blue, bold=true)
display(latexify([A', A_inv']'))
println()
printstyled( "rank $k Update", color=:blue, bold=true)
latexify( U*V')

A and A⁻¹

"\\begin{equation}\n\\left[\n\\begin{array}{cc}\n\\left[\n\\begin{array}{cccccccc}\n1 & 2 & 3 & 0 & 1 & -2 & -3 & -2 \\\\\n-1 & -1 & -5 & -1 & -2 & 5 & 5 & 2 \\\\\n-3 & -7 & -6 & 0 & -5 & 1 & 10 & 8 \\\\\n-1 & -3 & -3 & 4 & 8 & 5 & -4 & 0 \\\\\n-2 & -4 & -8 & 3 & 7 & 8 & 3 & 5 \\\\\n-3 & -4 & -11" ⋯ 281 bytes ⋯ "-1 & -9 \\\\\n-163 & -94 & -45 & -21 & 25 & 14 & 5 & 10 \\\\\n126 & 48 & 29 & 6 & -2 & -5 & -1 & -5 \\\\\n5 & 10 & 3 & 4 & -6 & -2 & -1 & -1 \\\\\n35 & -1 & 5 & -4 & 9 & 2 & 1 & 0 \\\\\n-62 & -9 & -11 & 3 & -9 & -1 & -1 & 1 \\\\\n\\end{array}\n\\right] \\\\\n\\end{array}\n\\right]\n\\end{equation}\n"


rank 3 Update

L"\begin{equation}
\left[
\begin{array}{cccccccc}
-6 & -8 & 6 & -4 & 10 & 10 & 8 & -8 \\
4 & 5 & -4 & 4 & -7 & -6 & -5 & 5 \\
12 & 17 & -10 & 8 & -23 & -20 & -19 & 19 \\
4 & 6 & -4 & 0 & -6 & -8 & -6 & 6 \\
6 & 6 & -8 & 8 & -8 & -8 & -4 & 4 \\
-12 & -19 & 8 & -4 & 25 & 22 & 23 & -23 \\
-6 & -8 & 6 & -4 & 10 & 10 & 8 & -8 \\
-24 & -28 & 28 & -24 & 36 & 36 & 24 & -24 \\
\end{array}
\right]
\end{equation}
"

In [5]:
S = I + V' * A_inv * U; S_inv = inv( S//1)
dS,intS=factor_out_denominator(S_inv)
printstyled( "Inverse of the k×k matrix", color=:blue, bold=true)
latexify([S', 1//dS, intS']')

Inverse of the k×k matrix

L"\begin{equation}
\left[
\begin{array}{ccc}
\left[
\begin{array}{ccc}
-2261 & -1676 & 1902 \\
944 & 1209 & -892 \\
-2294 & -1434 & 1887 \\
\end{array}
\right] & \frac{1}{9739975} & \left[
\begin{array}{ccc}
-1002255 & -435144 & 804526 \\
-264920 & -96681 & 221324 \\
-1419750 & -602470 & 1151405 \\
\end{array}
\right] \\
\end{array}
\right]
\end{equation}
"

In [6]:
@show inv(Rational{Int}.(A + U*V')) == (A_inv-A_inv*U*S_inv*V'*A_inv);

inv(Rational{Int}.(A + U * V')) == A_inv - A_inv * U * S_inv * V' * A_inv = true


___
Note the matrix update $U V^t$ need not have full column rank $k$:

In [7]:
_,U = gen_gj_matrix(M,k,1;maxint=2)
_,V = gen_gj_matrix(M,k,1;maxint=2)
printstyled( "rank 1 Update", color=:blue, bold=true)
latexify( U*V')
S = I + V' * A_inv * U; S_inv = inv( S//1)
dS,intS=factor_out_denominator(S_inv)
printstyled( "Inverse of the k×k matrix", color=:blue, bold=true)
display(latexify([S', 1//dS, intS']'))
@show inv(A + U*V'//1) == (A_inv-A_inv*U*S_inv*V'*A_inv);

rank 1 UpdateInverse of the k×k matrix

L"\begin{equation}
\left[
\begin{array}{ccc}
\left[
\begin{array}{ccc}
7545 & -3772 & -3772 \\
3772 & -1885 & -1886 \\
7544 & -3772 & -3771 \\
\end{array}
\right] & \frac{1}{1887} & \left[
\begin{array}{ccc}
-5657 & 3772 & 3772 \\
-3772 & 3773 & 1886 \\
-7544 & 3772 & 5659 \\
\end{array}
\right] \\
\end{array}
\right]
\end{equation}
"

inv(A + U * V' // 1) == A_inv - A_inv * U * S_inv * V' * A_inv = true


## 1.4. Solve $B x = b$ by Solving a Simpler System

Suppose we wish to solve $B x = b$, where $B = A + U V^t$ is close to some "nice" matrix $A$.

We can solve $B x = b$ as follows:<br>
$\qquad \begin{align}x = (A + U V^t)^{-1}\ b
& = A^{-1} b - A^{-1} U \left( I + V^t A^{-1} U \right)^{-1} V^t A^{-1} b \\
& = \tilde{x} - A^{-1} U \left( I + V^t A^{-1} U \right)^{-1} V^t \tilde{x} \\
\end{align}$<br>
$\qquad$ where $\tilde{x}$ is the solution to the "simpler" system $A \tilde{x} = b$.

$\qquad$ As usual, we will avoid computing the inverses explicitely.

* Step 1: Solve $A \tilde{x} = b$
* Step 2: Compute $W = A^{-1} U$ by solving $A W = U$
* Step 3: Compute $y = (I + V^t W)^{-1} V^t \tilde{x}$ by solving $(I+V^t W) y = V^t \tilde{x}$
* Step 4: Compute the solution  $x = \tilde{x} - W y$

In [8]:
# create a problem
M = 4
A = [-1 0 0 0; 0 -2 0 0; 2 -2 2 0; -1 -1 -1 3]
u = [2; 2; 3; -1]
v = [1; 2; 1; -3]
B = A+u*v'
b = [-5; -2; 2; 3]
printstyled( "Problem B x = b and simpler problem A x = b, where B, A, b =", color=:blue, bold=true)
display(latexify([B', A', b']'))

# step 1: Solve A x̃ = b
x̃ = A \ b
# step 2:  Compute W = A^{-1} U by solving A w_i = u_i, where w_i, and u_i, denote the ith column of W and U, respectively.
w = A \ u
# step 3: Solve (I+VᵗW) y = Vᵗx̃
y = (I + v'w) \ (v'x̃)
# step 4: Solve $x = x̃ - W y
x = x̃ - w*y
printstyled( "Solution of B x = b from the solution of A x = b", color=:blue, bold=true)
latexify([Int.(round.(x))', Int.(round.(x̃))']')

Problem B x = b and simpler problem A x = b, where B, A, b =

L"\begin{equation}
\left[
\begin{array}{ccc}
\left[
\begin{array}{cccc}
1 & 4 & 2 & -6 \\
2 & 2 & 2 & -6 \\
5 & 4 & 5 & -9 \\
-2 & -3 & -2 & 6 \\
\end{array}
\right] & \left[
\begin{array}{cccc}
-1 & 0 & 0 & 0 \\
0 & -2 & 0 & 0 \\
2 & -2 & 2 & 0 \\
-1 & -1 & -1 & 3 \\
\end{array}
\right] & \left[
\begin{array}{c}
-5 \\
-2 \\
2 \\
3 \\
\end{array}
\right] \\
\end{array}
\right]
\end{equation}
"

Solution of B x = b from the solution of A x = b

L"\begin{equation}
\left[
\begin{array}{cc}
\left[
\begin{array}{c}
1 \\
-1 \\
2 \\
1 \\
\end{array}
\right] & \left[
\begin{array}{c}
5 \\
1 \\
-3 \\
2 \\
\end{array}
\right] \\
\end{array}
\right]
\end{equation}
"

In [9]:
#check
@show (A*x̃ ≈ b) && (B*x ≈ b);

A * x̃ ≈ b && B * x ≈ b = true


## 1.5 Updating a Regression Solution

Consider the regression problem $argmin_x \Vert A x - b \Vert$<br>
$\qquad$ which is solved by the normal equation $A^t A x = A^t b$.

We wish to update this solution when we get one or more new measurements $r, \tilde{b}$, i.e.,<br>
$\qquad$ we now want the solution of
$\begin{pmatrix} A^t & r^t \end{pmatrix} \begin{pmatrix} A \\ r \end{pmatrix} \tilde{x}
= \begin{pmatrix} A^t & r^t \end{pmatrix} \begin{pmatrix} b \\ \tilde{b} \end{pmatrix}\;\;\Leftrightarrow \;\;
(A^t A + r^t r) \tilde{x} =A^t b + r^t\tilde{b}$

Setting $D = I,\; A = B,\; U=r^t$ and $V=r$ in Eq 3, we obtain

$\qquad (B + r^t r)^{-1} = \left( I - W \left( I + r W \right)^{-1} r \right) B^{-1},\;\;$ where $\;\;W=B^{-1} r^t,\;$
and $B = A^t A$


Applying this equation to the right hand side $A^t b + r^t \tilde{b}$, the updated solution $\tilde{x}$ of the normal equation is given by<br>
$\qquad\tilde{x} = \left( I - W \left( I + r W \right)^{-1} r \right) \left( x + W \tilde{b} \right)$


In the case where $r$ is size $1 \times N$, i.e., a single measurement, this reduces to<br>
$\qquad \tilde{x} = \left( I - \frac{1}{\gamma} W r \right) \left( x + W\tilde{b}\right)$, where 
$\gamma = 1 + r W$.

#### Example

In [10]:
# Set up a problem AᵗA x = Aᵗ b and an update 
model(x) = 2 .+ 3x
M = 20; k = 5
Ã = [ones(M+k) 1:(M+k)]
A = Ã[1:M,1:end]
r = Ã[M+1:end,1:end]
b = model(A[:,2])+randn(M)
b̃ = model(r[:,2])+randn(k)

# Solve the normal equation and the updated normal equation
x     = (A'A) \ (A'b)
x_new = (Ã'Ã) \ (Ã'*[b;b̃])

printstyled("Estimate:     y = $(round(x[1],digits=3)) + $(round(x[2],digits=3)) x\n", color=:blue, bold=true)
printstyled("New Estimate: y = $(round(x_new[1],digits=3)) + $(round(x_new[2],digits=3)) x", color=:blue, bold=true)

Estimate:     y = 1.382 + 3.034 x
New Estimate: y = 1.36 + 3.036 x

In [11]:
# update the previous solution instead:
B = A'A                         # the original Gram Matrix
W = B \ r'
y = (I+r*W) \ r
x̃ = (I-W*y)*(x+ W*b̃)            # the updated solution
printstyled("New Estimate: y = $(round(x̃[1],digits=3)) + $(round(x̃[2],digits=3)) x", color=:blue, bold=true)

New Estimate: y = 1.36 + 3.036 x

# 2. Other Useful Formulae

The following formulae commute terms $A B$ to terms $B A$. These equalities follow from $B ( A B ) = ( B A ) B$.

\begin{aligned}
& B \left( I_m + A B \right)  & = &\quad \left(I_n + B A \right) B \\
& B \left( I_m + A B \right)^{-1} & = &\quad \left(I_n + B A \right)^{-1} B \\
& A^t \left( A A^t + \lambda I_n \right)^{-1} & = &\quad \left(A^t A + \lambda I_m \right)^{-1} A^t \\
& U \left( I_k + V^t U \right) & = &\quad \left(I_n + U V^t \right) U \\
\end{aligned}

We also have
$$ B^{-1} - A^{-1} = B^{-1} ( A - B ) A^{-1}$$